In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets
from torchvision import transforms
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as tutils
import copy
from PIL import Image

In [2]:
batch_size = 100
imgDim = 28
path = './genImg/'

transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=(0.5, 0.5, 0.5), 
                                     std=(0.5, 0.5, 0.5))])

In [3]:
# MNIST dataset
dataset = datasets.MNIST(root='../Data/MNIST',
                         train=True,
                         transform=transform,
                         download=True)
# Data loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                          batch_size=batch_size, 
                                          shuffle=True)


In [4]:
IS_CUDA = False
if torch.cuda.is_available():
    IS_CUDA = True
    
def var(x):
    if IS_CUDA:
        x = x.cuda()
    return Variable(x)

def show(img):
    npimg = img.data.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)), interpolation='nearest')
    
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def save_image(pic, path):
    grid = torchvision.utils.make_grid(pic.data, nrow=8, padding=2)
    ndarr = grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).cpu().numpy()
    im = Image.fromarray(ndarr)
    im.save(path)

In [5]:
Generator_input = 64
'''
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = x.resize(batch_size, 1, imgDim, imgDim)
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = self.conv2(x)
        x = F.relu(F.max_pool2d(self.conv2_drop(x),2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        return F.sigmoid(x)
'''
    
    
D = nn.Sequential(
        nn.Linear(784, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256,1),
        nn.Sigmoid())

Generator_input = 64

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(Generator_input, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Sequential(nn.Linear(256, 784), nn.Tanh())
        self.lR = nn.LeakyReLU(0.2)
            
    def forward(self, x):
        x = self.fc1(x)
        x = self.lR(x)
        x = self.fc2(x)
        x = self.lR(x)
        return self.fc3(x)
'''

Generator = nn.Sequential(
        nn.Linear(Generator_input, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 256),
        nn.LeakyReLU(0.2),
        nn.Linear(256, 784),
        nn.Tanh())

'''
#D = Discriminator()

# Create n Generators
#Gen = copy.deepcopy(Generator)
Gen = Generator()
Generators = []
GOptimizers = []
NumberOfGenerators = 3
for i in range(NumberOfGenerators):
    #Generators.append(copy.deepcopy(Generator))
    Generators.append(Generator())

if IS_CUDA:
    D.cuda()
    for each in Generators:
        each.cuda()

In [6]:
lossCriterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters(), lr = 0.0001)
G_opt = torch.optim.Adam(Gen.parameters(), lr = 0.0001)
GOptimizers = []
for each in Generators:
    GOptimizers.append(torch.optim.Adam(each.parameters(), lr = 0.0001))

fixed_x = var(torch.randn(batch_size, Generator_input))

GeneratorLoss = []
def backPropGenerator(index, GeneratorLoss):
    Generators[index].zero_grad()
    GeneratorLoss[index].backward()
    GOptimizers[index].step()



In [7]:
'''
outputImages = []
BestPerformingGenerator = 0
def train(Gen, BestPerformingGenerator, num_epochs = 10, d_iter = 1):
    for epoch in range(num_epochs):
        for data in data_loader:
            image, _  = data
            image = var(image.view(image.size(0),  -1))
            
            #Gen = copy.deepcopy(Generators[BestPerformingGenerator])
            
            # Train Discriminator
            #for k in range(0, d_iter):
            # For Log D(x)
            #print image.shape
            D_real = D(image)
            # For Log(1 - D(G(Z)))
            Z_noise = var(torch.randn(batch_size, Generator_input))
            #print Z_noise.shape
            #print type(Gen)
            G_fake = Gen(Z_noise)
            #print G_fake.shape
            D_fake = D(G_fake)

            # Calculate Discriminator Loss
            D_real_loss = lossCriterion(D_real, var(torch.ones(batch_size, 1)))
            D_fake_loss = lossCriterion(D_fake, var(torch.zeros(batch_size, 1)))
            D_loss = D_real_loss + D_fake_loss

            # Backprop Discriminator
            D.zero_grad()
            D_loss.backward()
            D_opt.step()
   
            # Train Generators
            Z_noise = var(torch.randn(batch_size, Generator_input))
            G_fake = Gen(Z_noise)
            D_fake = D(G_fake)
            # Compute Generator Loss
            G_loss = lossCriterion(D_fake, var(torch.ones(batch_size, 1)))
            
            # Find best performing Generator
            GeneratorLoss = []
            lossList = []
            for each in Generators:
                Z_noise1 = var(torch.randn(batch_size, Generator_input))
                G_fake1 = each(Z_noise1)
                #print G_fake1.shape
                #print type(each)
                D_fake1 = D(G_fake1)
                # Compute Generator Loss
                G_loss1 = lossCriterion(D_fake1, var(torch.ones(batch_size, 1)))
                GeneratorLoss.append(G_loss1)
                lossList.append(float(G_loss1.data[0]))
            
            #print lossList
            #print type(lossList[0])
            BestPerformingGenerator = lossList.index(min(lossList))
            
            # Change other Generator with best performing ones config:
            Gen = copy.deepcopy(Generators[BestPerformingGenerator])
            G_opt = copy.deepcopy(GOptimizers[BestPerformingGenerator])

            # Backprop Genearator
            D.zero_grad()
            Gen.zero_grad()
            G_loss.backward()
            G_opt.step()
            
            backPropGenerator(BestPerformingGenerator, GeneratorLoss)
            
            for i in range(NumberOfGenerators):
                if i != BestPerformingGenerator:
                    Generators[i] = copy.deepcopy(Generators[BestPerformingGenerator])
                    GOptimizers[i] = copy.deepcopy(GOptimizers[BestPerformingGenerator])
                    backPropGenerator(i, GeneratorLoss)
                                
        #print epoch
        print 'Epoch [{}/{}], Discriminator {:.4f}, Generator {:.4f}'.format(epoch+1, num_epochs, D_loss.data[0], G_loss.data[0])
        pic = Gen(var(torch.randn(batch_size, Generator_input))) #(fixed_x)
        pic = pic.view(pic.size(0), 1, imgDim, imgDim) 
        outputImages.append(pic)
        torchvision.utils.save_image(pic.data, path+'image_{}.png'.format(epoch)) 
'''

"\noutputImages = []\nBestPerformingGenerator = 0\ndef train(Gen, BestPerformingGenerator, num_epochs = 10, d_iter = 1):\n    for epoch in range(num_epochs):\n        for data in data_loader:\n            image, _  = data\n            image = var(image.view(image.size(0),  -1))\n            \n            #Gen = copy.deepcopy(Generators[BestPerformingGenerator])\n            \n            # Train Discriminator\n            #for k in range(0, d_iter):\n            # For Log D(x)\n            #print image.shape\n            D_real = D(image)\n            # For Log(1 - D(G(Z)))\n            Z_noise = var(torch.randn(batch_size, Generator_input))\n            #print Z_noise.shape\n            #print type(Gen)\n            G_fake = Gen(Z_noise)\n            #print G_fake.shape\n            D_fake = D(G_fake)\n\n            # Calculate Discriminator Loss\n            D_real_loss = lossCriterion(D_real, var(torch.ones(batch_size, 1)))\n            D_fake_loss = lossCriterion(D_fake, var(torch.

In [12]:
#train(Gen, 0, 10)

[0, 0, 0]


In [7]:
outputImages = []
def train(BestPerformingGenerator, num_epochs = 10, d_iter = 1):
    for epoch in range(num_epochs):
        lossList = [0.0] * NumberOfGenerators
        for data in data_loader:
            image, _  = data
            image = var(image.view(image.size(0),  -1))
            
            #Gen = copy.deepcopy(Generators[BestPerformingGenerator])
            
            # Train Discriminator
            #for k in range(0, d_iter):
            # For Log D(x)
            #print image.shape
            #print 'Epoch [{}/{}].'.format(epoch+1, num_epochs)
            
            #for each in Generators:
            D_real = D(image)
            # For Log(1 - D(G(Z)))
            Z_noise = var(torch.randn(batch_size, Generator_input))
            #print Z_noise.shape
            #print type(Gen)
            G_fake = Generators[BestPerformingGenerator](Z_noise)
            #print G_fake.shape
            D_fake = D(G_fake)

            # Calculate Discriminator Loss
            D_real_loss = lossCriterion(D_real, var(torch.ones(batch_size, 1)))
            D_fake_loss = lossCriterion(D_fake, var(torch.zeros(batch_size, 1)))
            D_loss = D_real_loss + D_fake_loss

            # Backprop Discriminator
            D.zero_grad()
            D_loss.backward()
            D_opt.step()
                #print 'Discriminator Loop for: {}: {}'.format(i, D_loss.data[0])
   
            # Find best performing Generator
            i = 0
            GeneratorLoss = []
            for each, each_opt in zip(Generators, GOptimizers):
                Z_noise = var(torch.randn(batch_size, Generator_input))
                G_fake = each(Z_noise)
                #print G_fake1.shape
                #print type(each)
                D_fake = D(G_fake)
                # Compute Generator Loss
                G_loss = lossCriterion(D_fake, var(torch.ones(batch_size, 1)))
                GeneratorLoss.append(G_loss)
                lossList[i] += (float(G_loss.data[0]))
                i = i + 1
                D.zero_grad()
                each.zero_grad()
                G_loss.backward()
                each_opt.step()
                
                #backPropGenerator(i, GeneratorLoss)
                #print 'Generator Loop for: {}: {}'.format(i, G_loss.data[0])
            
            #print lossList
            #print type(lossList[0])
        BestPerformingGenerator = lossList.index(max(lossList)) # earlier was min
        print lossList
        for i in range(0, NumberOfGenerators):
            if i != BestPerformingGenerator:
                prev = Generators[i]
                Generators[i] = copy.deepcopy(Generators[BestPerformingGenerator])
                GOptimizers[i] = torch.optim.Adam(Generators[i].parameters(), lr = 0.0001)
                #copy.deepcopy(GOptimizers[BestPerformingGenerator])
                if Generators[i] == prev:
                    print 'SAME'

        #print epoch
        #print BestPerformingGenerator
        print 'Epoch [{}/{}], Discriminator {:.4f}, Best Generator[{}] {:.4f}'.format(epoch+1, num_epochs, D_loss.data[0], BestPerformingGenerator, GeneratorLoss[BestPerformingGenerator].data[0])
        pic = Generators[BestPerformingGenerator](var(torch.randn(batch_size, Generator_input))) #(fixed_x)
        pic = pic.view(pic.size(0), 1, imgDim, imgDim) 
        outputImages.append(pic)
        #torchvision.utils.save_image(pic.data, path+'image_{}.png'.format(epoch))   
        save_image(pic, path+'image_{}.png'.format(epoch))

In [7]:
# Train D for every G

outputImages = []
def train(BestPerformingGenerator, num_epochs = 10, d_iter = 1):
    for epoch in range(num_epochs):
        lossList = [0.0] * NumberOfGenerators
        for data in data_loader:
            image, _  = data
            image = var(image.view(image.size(0),  -1))
            
            #Gen = copy.deepcopy(Generators[BestPerformingGenerator])
            
            # Train Discriminator
            #for k in range(0, d_iter):
            # For Log D(x)
            #print image.shape
            #print 'Epoch [{}/{}].'.format(epoch+1, num_epochs)
            
            for each in Generators:
                D_real = D(image)
                # For Log(1 - D(G(Z)))
                Z_noise = var(torch.randn(batch_size, Generator_input))
                #print Z_noise.shape
                #print type(Gen)
                G_fake = each(Z_noise) #Generators[BestPerformingGenerator](Z_noise)
                #print G_fake.shape
                D_fake = D(G_fake)

                # Calculate Discriminator Loss
                D_real_loss = lossCriterion(D_real, var(torch.ones(batch_size, 1)))
                D_fake_loss = lossCriterion(D_fake, var(torch.zeros(batch_size, 1)))
                D_loss = D_real_loss + D_fake_loss

                # Backprop Discriminator
                D.zero_grad()
                D_loss.backward()
                D_opt.step()
                #print 'Discriminator Loop for: {}: {}'.format(i, D_loss.data[0])
   
            # Find best performing Generator
            i = 0
            GeneratorLoss = []
            for each, each_opt in zip(Generators, GOptimizers):
                Z_noise = var(torch.randn(batch_size, Generator_input))
                G_fake = each(Z_noise)
                #print G_fake1.shape
                #print type(each)
                D_fake = D(G_fake)
                # Compute Generator Loss
                G_loss = lossCriterion(D_fake, var(torch.ones(batch_size, 1)))
                GeneratorLoss.append(G_loss)
                lossList[i] += (float(G_loss.data[0]))
                i = i + 1
                D.zero_grad()
                each.zero_grad()
                G_loss.backward()
                each_opt.step()
                
                #backPropGenerator(i, GeneratorLoss)
                #print 'Generator Loop for: {}: {}'.format(i, G_loss.data[0])
            
            #print lossList
            #print type(lossList[0])
        BestPerformingGenerator = lossList.index(max(lossList)) # earlier was min
        print lossList
        for i in range(0, NumberOfGenerators):
            if i != BestPerformingGenerator:
                prev = Generators[i]
                Generators[i] = copy.deepcopy(Generators[BestPerformingGenerator])
                GOptimizers[i] = torch.optim.Adam(Generators[i].parameters(), lr = 0.0001)
                #copy.deepcopy(GOptimizers[BestPerformingGenerator])
                if Generators[i] == prev:
                    print 'SAME'

        #print epoch
        #print BestPerformingGenerator
        print 'Epoch [{}/{}], Discriminator {:.4f}, Best Generator[{}] {:.4f}'.format(epoch+1, num_epochs, D_loss.data[0], BestPerformingGenerator, GeneratorLoss[BestPerformingGenerator].data[0])
        pic = Generators[BestPerformingGenerator](var(torch.randn(batch_size, Generator_input))) #(fixed_x)
        pic = pic.view(pic.size(0), 1, imgDim, imgDim) 
        outputImages.append(pic)
        #torchvision.utils.save_image(pic.data, path+'image_{}.png'.format(epoch))   
        save_image(pic, path+'image_{}.png'.format(epoch))

In [8]:
# load model instead of deepcopy

outputImages = []
def train(BestPerformingGenerator, num_epochs = 10, d_iter = 1):
    for epoch in range(num_epochs):
        lossList = [0.0] * NumberOfGenerators
        for data in data_loader:
            image, _  = data
            image = var(image.view(image.size(0),  -1))
            
            #Gen = copy.deepcopy(Generators[BestPerformingGenerator])
            
            # Train Discriminator
            #for k in range(0, d_iter):
            # For Log D(x)
            #print image.shape
            #print 'Epoch [{}/{}].'.format(epoch+1, num_epochs)
            
            for each in Generators:
                D_real = D(image)
                # For Log(1 - D(G(Z)))
                Z_noise = var(torch.randn(batch_size, Generator_input))
                #print Z_noise.shape
                #print type(Gen)
                G_fake = each(Z_noise) #Generators[BestPerformingGenerator](Z_noise)
                #print G_fake.shape
                D_fake = D(G_fake)

                # Calculate Discriminator Loss
                D_real_loss = lossCriterion(D_real, var(torch.ones(batch_size, 1)))
                D_fake_loss = lossCriterion(D_fake, var(torch.zeros(batch_size, 1)))
                D_loss = D_real_loss + D_fake_loss

                # Backprop Discriminator
                D.zero_grad()
                D_loss.backward()
                D_opt.step()
                #print 'Discriminator Loop for: {}: {}'.format(i, D_loss.data[0])
   
            # Find best performing Generator
            i = 0
            GeneratorLoss = []
            for each, each_opt in zip(Generators, GOptimizers):
                Z_noise = var(torch.randn(batch_size, Generator_input))
                G_fake = each(Z_noise)
                #print G_fake1.shape
                #print type(each)
                D_fake = D(G_fake)
                # Compute Generator Loss
                G_loss = lossCriterion(D_fake, var(torch.ones(batch_size, 1)))
                GeneratorLoss.append(G_loss)
                lossList[i] += (float(G_loss.data[0]))
                i = i + 1
                D.zero_grad()
                each.zero_grad()
                G_loss.backward()
                each_opt.step()
                
                #backPropGenerator(i, GeneratorLoss)
                #print 'Generator Loop for: {}: {}'.format(i, G_loss.data[0])
            
            #print lossList
            #print type(lossList[0])
        BestPerformingGenerator = lossList.index(max(lossList)) # earlier was min
        print lossList
        for i in range(0, NumberOfGenerators):
            if i != BestPerformingGenerator:
                prev = Generators[i]
                Generators[i] = Generator()
                Generators[i].load_state_dict(Generators[BestPerformingGenerator].state_dict())
                GOptimizers[i] = torch.optim.Adam(Generators[i].parameters(), lr = 0.0001)
                GOptimizers[i].load_state_dict(GOptimizers[BestPerformingGenerator].state_dict())
                #Generators[i] = copy.deepcopy(Generators[BestPerformingGenerator])
                #GOptimizers[i] = torch.optim.Adam(Generators[i].parameters(), lr = 0.0001)
                #copy.deepcopy(GOptimizers[BestPerformingGenerator])
                if Generators[i] == prev:
                    print 'SAME'

        #print epoch
        #print BestPerformingGenerator
        print 'Epoch [{}/{}], Discriminator {:.4f}, Best Generator[{}] {:.4f}'.format(epoch+1, num_epochs, D_loss.data[0], BestPerformingGenerator, GeneratorLoss[BestPerformingGenerator].data[0])
        pic = Generators[BestPerformingGenerator](var(torch.randn(batch_size, Generator_input))) #(fixed_x)
        pic = pic.view(pic.size(0), 1, imgDim, imgDim) 
        outputImages.append(pic)
        #torchvision.utils.save_image(pic.data, path+'image_{}.png'.format(epoch))   
        torchvision.utils.save_image(denorm(pic.data), path+'images-%d.png' %(epoch+1))


In [9]:
#train(0, 30)

In [ ]:
# Load model instead of deep copy
train(0, 30)

[2532.735988497734, 2510.1840450763702, 2678.7209434509277]
Epoch [1/30], Discriminator 0.0544, Best Generator[2] 4.8398
[3708.898733139038, 3699.0548515319824, 3691.1821489334106]
Epoch [2/30], Discriminator 0.0139, Best Generator[0] 6.0457
[2978.1063261032104, 2990.697278857231, 2994.3041499853134]
Epoch [3/30], Discriminator 0.0144, Best Generator[2] 5.9195
[2862.3172655701637, 2726.8054524064064, 2686.2068423628807]
Epoch [4/30], Discriminator 0.1163, Best Generator[0] 4.1751
[3176.44689142704, 3026.2978740036488, 3353.713204741478]
Epoch [5/30], Discriminator 0.1805, Best Generator[2] 3.0685
[3912.1018545627594, 4083.277430295944, 3572.6396095752716]
Epoch [6/30], Discriminator 0.1003, Best Generator[1] 4.9545
[4688.705378457904, 4699.2129918932915, 4705.606785058975]
Epoch [7/30], Discriminator 0.3109, Best Generator[2] 10.4148
[5081.209809780121, 4477.288584530354, 5128.331526756287]
Epoch [8/30], Discriminator 0.1063, Best Generator[2] 3.9163
[4918.330916404724, 5007.8437178134

In [9]:
# With optimizer on the fly
train(0, 40)

[856.9920598268509, 803.1024285107851, 851.8648067414761]
Epoch [1/40], Discriminator 0.4304, Best Generator[1] 4.6023
[1054.6738081872463, 1334.7822082042694, 1052.1758828014135]
Epoch [2/40], Discriminator 0.9626, Best Generator[2] 0.2424
[1736.5196833498776, 1736.0395818315446, 1815.742018699646]
Epoch [3/40], Discriminator 0.3986, Best Generator[1] 0.2782
[1382.5161429196596, 1580.1552543640137, 1371.2793620675802]
Epoch [4/40], Discriminator 0.9156, Best Generator[2] 0.3780
[1281.1298932731152, 1282.6176551878452, 1666.1951867341995]
Epoch [5/40], Discriminator 0.3574, Best Generator[0] 0.1930
[1786.1965844631195, 870.0297492556274, 869.3980573303998]
Epoch [6/40], Discriminator 0.3847, Best Generator[2] 0.1438
[843.1533146132715, 844.1812381210038, 1685.6768636703491]
Epoch [7/40], Discriminator 0.7598, Best Generator[0] 0.0019
[1026.7727580076316, 272.8395392104285, 274.05790696816985]
Epoch [8/40], Discriminator 0.4641, Best Generator[1] 0.1701
[1738.527152934541, 1527.20318979

KeyboardInterrupt: 

In [8]:
# With optimizer on the fly and max loss
train(0, 30)

[872.1570700109005, 662.0861651301384, 609.2562948316336]
Epoch [1/30], Discriminator 0.6847, Best Generator[0] 1.7367
[814.5087671279907, 751.060758471489, 748.6468422412872]
Epoch [2/30], Discriminator 1.0386, Best Generator[0] 1.1394
[1001.8705675601959, 402.21531569585204, 399.4936091527343]
Epoch [3/30], Discriminator 0.8658, Best Generator[0] 2.0115
[1254.4419853687286, 850.1604254692793, 845.96289460361]
Epoch [4/30], Discriminator 0.4126, Best Generator[0] 2.5391
[1661.9885839223862, 1183.961763634812, 1223.4222673280165]
Epoch [5/30], Discriminator 0.4755, Best Generator[0] 3.0569
[1639.9664514064789, 1637.8508403003216, 1646.1216702461243]
Epoch [6/30], Discriminator 0.5279, Best Generator[2] 0.4772
[1645.3805298805237, 1632.3828236460686, 1544.1907473802567]
Epoch [7/30], Discriminator 0.4334, Best Generator[0] 0.8038
[1624.6379365324974, 1345.9067988991737, 1343.315298974514]
Epoch [8/30], Discriminator 0.5002, Best Generator[0] 2.7455
[2061.575703859329, 1457.665353178978,

In [8]:
train(0, 30)

[2778.058117568493, 2917.6230896115303, 2697.726538479328]
Epoch [1/30], Discriminator 0.0416, Best Generator[1] 5.6720
[3695.9072647094727, 3356.031274318695, 3374.9563450813293]
Epoch [2/30], Discriminator 0.1799, Best Generator[0] 6.3682
[3256.957720041275, 3510.96928858757, 3557.807953119278]
Epoch [3/30], Discriminator 0.0258, Best Generator[2] 6.8425
[4142.80516242981, 4470.754581928253, 3614.0237057209015]
Epoch [4/30], Discriminator 0.0879, Best Generator[1] 6.4516
[4529.959128856659, 3948.716283798218, 4235.133533000946]
Epoch [5/30], Discriminator 0.0387, Best Generator[0] 8.0247
[4343.090248584747, 4722.502594470978, 4257.555803775787]
Epoch [6/30], Discriminator 0.2769, Best Generator[1] 5.8655
[4618.889126300812, 4463.452397823334, 4359.163161277771]
Epoch [7/30], Discriminator 0.2528, Best Generator[0] 6.2346
[4684.915420532227, 4336.271741390228, 4291.608334541321]
Epoch [8/30], Discriminator 0.1100, Best Generator[0] 7.7472
[4734.094176769257, 4796.5427713394165, 4795.7

In [9]:
#torch.save(G.state_dict(), './Generator.pkl')
#torch.save(D.state_dict(), './Discriminator.pkl')

In [ ]:
#Generator.load_state_dict(torch.load('Generator200.pkl'))
#Discriminator.load_state_dict(torch.load('Discriminator200.pkl'))

In [ ]:
#import imageio
#imageio.mimsave(path + 'MNIST_MaxNew_{:d}'.format(30) + '.gif', outputImages, fps=5)